In [95]:
import pandas as pd
import re
from urllib.parse import urlparse, unquote
from geopy.geocoders import Nominatim

# Limpieza de datos

Los datos procedentes del scraping tienen formatos que no nos interesan (por ejemplo el símbolo euro después de cada precio o 'kms' después de cada distancia). Además hay bastantes NaN que tenemos que tratar. Por eso, antes de pasar a las fase de análisis y de machine learning será necesario que limpiemos los datos.

In [2]:
df=pd.read_csv("vehiculos_usados.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11173 entries, 0 to 11172
Data columns (total 50 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Unnamed: 0                            11173 non-null  int64  
 1   Fecha extrac                          11173 non-null  object 
 2   Enlace                                11173 non-null  object 
 3   Marca                                 11173 non-null  object 
 4   Modelo                                10768 non-null  object 
 5   Precio                                11173 non-null  object 
 6   Localización                          11173 non-null  object 
 7   Potencia                              11173 non-null  object 
 8   Tipo vendedor                         11173 non-null  object 
 9   Categoría                             11171 non-null  object 
 10  Tipo de vehículo                      11171 non-null  object 
 11  puertas        

In [3]:
df.isna().sum()

Unnamed: 0                                  0
Fecha extrac                                0
Enlace                                      0
Marca                                       0
Modelo                                    405
Precio                                      0
Localización                                0
Potencia                                    0
Tipo vendedor                               0
Categoría                                   2
Tipo de vehículo                            2
puertas                                    78
Versión del país                          465
Núm. de oferta                            465
Garantía                                 2302
Kilometraje                               569
Año                                       569
Tipo de cambio                             89
Capacidad                                1323
Otras fuentes de energía                10226
Consumo de combustible\n2                1926
Color exterior                    

Como podemos ver hay columnas que tienen más de la mitad de los registros en NaN, en concreto las últimas 21 columnas. Así que empezaremos eliminando esas.

In [4]:
df = df.iloc[:,:-21]
df.isna().sum()

Unnamed: 0                       0
Fecha extrac                     0
Enlace                           0
Marca                            0
Modelo                         405
Precio                           0
Localización                     0
Potencia                         0
Tipo vendedor                    0
Categoría                        2
Tipo de vehículo                 2
puertas                         78
Versión del país               465
Núm. de oferta                 465
Garantía                      2302
Kilometraje                    569
Año                            569
Tipo de cambio                  89
Capacidad                     1323
Otras fuentes de energía     10226
Consumo de combustible\n2     1926
Color exterior                1290
Color original                3714
Tracción                      2687
plazas                        2258
Número de marchas             2489
Número de cilindros           2732
Peso                          3496
Tipo de combustible 

Nos sigue quedando el atributo 'Otras fuentes de energía' con la mayoría de valores en NaN. Lo eliminamos también.

In [5]:
df=df.drop(['Otras fuentes de energía'], axis=1)
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior               1290
Color original               3714
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros          2732
Peso                         3496
Tipo de combustible           986
dtype: int64

Vamos a ocuparnos ahora de los NaN de los atributos 'Color original' y 'Color exterior'

In [6]:
df.loc[:50, ['Color exterior', 'Color original']]

,Color exterior,Color original
0,Azul,Azul
1,Negro,Negro Midnight (metalizado)
2,Gris,NaN
3,NaN,ROJO
4,Plateado,Gris Plata
5,Gris,Gris Ágata
6,Rojo,Rosso Corsa
7,Negro,Negro
8,Blanco,Blanco
9,Azul,PACK AMG


Asumiremos que la mayoría de coches no son repintados para cambiarles el color. De manera que cuando haya un NaN en el color original le copiaremos el valor del color exterior y viceversa.

In [7]:
df['Color exterior'].fillna(df['Color original'], inplace=True)
df['Color original'].fillna(df['Color exterior'], inplace=True)

In [8]:
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                704
Color original                704
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros          2732
Peso                         3496
Tipo de combustible           986
dtype: int64

Con esta maniobra hemos reducido los NaN de la variable 'Color exterior' a casi la mitad y los de la variable 'Color original' en un 80%. No creemos que el color sea un dato tan relevante como para justificar eliminar 640 registros, así que lo rellenaremos con la moda.

In [9]:
print("La moda de 'Color exterior' es", df['Color exterior'].mode()[0])
print("La moda de 'Color original' es", df['Color original'].mode()[0])

La moda de 'Color exterior' es Blanco
La moda de 'Color original' es Blanco


In [10]:
df['Color exterior'].fillna(df['Color exterior'].mode()[0], inplace=True)
df['Color original'].fillna(df['Color original'].mode()[0], inplace=True)
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros          2732
Peso                         3496
Tipo de combustible           986
dtype: int64

Ahora vamos a ocuparnos de los NaN del atributo Peso. Primero intentaremos ver si los podemos rellenar cogiendo el peso de los coches de la misma marca y modelo que lo tengan indicado.

In [11]:
df['Peso'] = df.groupby(['Marca', 'Modelo'])['Peso'].fillna(method='ffill')

In [12]:
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros          2732
Peso                         1678
Tipo de combustible           986
dtype: int64

Así hemos conseguido completar algo más de la mitad de los valores NaN. Vamos a ver si completando de atrás hacia alante conseguimos completar alguno más.

In [13]:
df['Peso'] = df.groupby(['Marca', 'Modelo'])['Peso'].fillna(method='bfill')
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros          2732
Peso                          771
Tipo de combustible           986
dtype: int64

Parece que ya no hay más coches de la misma marca y modelo con los que completar la información. Para los restantes queremos ponerle la media de peso de los coches de esa misma marca. Pero no podemos calcularla aún porque el peso es un texto con el sufijo 'kg'. Primero tendremos que quitarle el sufijo y cambiarle el tipo al atributo 'Peso'

In [14]:
def process_numbers_and_nans(text):
    if pd.isna(text):
        return None
    numbers = re.findall(r'\d+', text)
    return int(''.join(numbers))

In [15]:
df['Peso']=df['Peso'].apply(process_numbers_and_nans)
df['Peso'].head()

0    1237.0
1    1761.0
2    1760.0
3    2215.0
4    1580.0
Name: Peso, dtype: float64

Ahora vamos a aplicar la media de la marca a los pesos que siguen en Nan

In [16]:
df['Peso'] = df.groupby(['Marca'])['Peso'].transform(lambda x: x.fillna(x.mean()))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros          2732
Peso                            6
Tipo de combustible           986
dtype: int64

Los que quedan en Nan deben ser los que no tienen otros vehículos con la misma marca con peso asignado. Así que los rellenaremos con la media de peso de coches, sin filtrar por marca

In [17]:
df['Peso'] = df['Peso'].transform(lambda x: x.fillna(x.mean()))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros          2732
Peso                            0
Tipo de combustible           986
dtype: int64

Repetiremos el proceso para el número de cilindros. Primero intentaremos llenar hacia alante y hacia atrás con los coches de la misma marca y modelo.

In [18]:
df['Número de cilindros'] = df.groupby(['Marca', 'Modelo'])['Número de cilindros'].fillna(method='ffill')
df['Número de cilindros'] = df.groupby(['Marca', 'Modelo'])['Número de cilindros'].fillna(method='bfill')
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros           537
Peso                            0
Tipo de combustible           986
dtype: int64

Hemos reducido significativamente el número de NaN. Ahora aplicaremos la media para esa marca a los que no hemos podido rellenar.

In [19]:
df['Número de cilindros'] = df.groupby(['Marca'])['Número de cilindros'].transform(lambda x: x.fillna(x.mean()))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros             6
Peso                            0
Tipo de combustible           986
dtype: int64

Y por último los que faltan les asignaremos la media de cilindros de todos nuestros registros.

In [20]:
df['Número de cilindros'] = df['Número de cilindros'].transform(lambda x: x.fillna(x.mean()))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                     2687
plazas                       2258
Número de marchas            2489
Número de cilindros             0
Peso                            0
Tipo de combustible           986
dtype: int64

Como no puede haber decimales en el número de cilindros que contiene un coche lo cambiaremos a entero.

In [21]:
df['Número de cilindros']=df['Número de cilindros'].astype('int64')

Repetiremos el mismo proceso con el número de marchas en el mismo orden. Rellenado con valores de la misma marca y modelo hacia alante y hacia atrás, media de la marca y media de de los coches

In [22]:
df['Número de marchas'] = df.groupby(['Marca', 'Modelo'])['Número de marchas'].fillna(method='ffill')
df['Número de marchas'] = df.groupby(['Marca', 'Modelo'])['Número de marchas'].fillna(method='bfill')
df['Número de marchas'] = df.groupby(['Marca'])['Número de marchas'].transform(lambda x: x.fillna(x.mean()))
df['Número de marchas'] = df['Número de marchas'].transform(lambda x: x.fillna(x.mean()))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                     2687
plazas                       2258
Número de marchas               0
Número de cilindros             0
Peso                            0
Tipo de combustible           986
dtype: int64

Cambiaremos la columna a entera porque no puede haber número de marchas decimales

In [23]:
df['Número de marchas']=df['Número de marchas'].astype('int64')
df['Número de marchas'].value_counts()

Número de marchas
6     4598
5     2038
8     1780
7     1426
1      729
9      337
4      174
3       47
2       31
10      13
Name: count, dtype: int64

Mismo proceso para el atributo 'Tracción' pero al no tratarse de un atributo numérico, para los dos últimos pasos usaremso la moda en lugar de la media.

In [24]:
df['Tracción'] = df.groupby(['Marca', 'Modelo'])['Tracción'].fillna(method='ffill')
df['Tracción'] = df.groupby(['Marca', 'Modelo'])['Tracción'].fillna(method='bfill')
#El siguiente bloque está con try/except porque si la moda de la marca era NaN crasheaba.
try:
    df['Tracción'] = df.groupby(['Marca'])['Tracción'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df['Tracción'] = df['Tracción'].transform(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                     2302
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                        0
plazas                       2258
Número de marchas               0
Número de cilindros             0
Peso                            0
Tipo de combustible           986
dtype: int64

Pasamos al atributo 'Garantía'. Lo primero que haremos es quitar el sufijo 'mes' que aparece después de cada valor.

In [25]:
df['Garantía']=df['Garantía'].str.extract(r'(\d+)')
df['Garantía'].head(20)

0      12
1      12
2      12
3      12
4      12
5      18
6      12
7      12
8      12
9      12
10     24
11     12
12     24
13    NaN
14    NaN
15     12
16     36
17    NaN
18     12
19     12
Name: Garantía, dtype: object

En España la legislación dice que los vendedores particulares deben ofrecer 6 meses de garantía para cubrir las averías ocultas que un vehículo de segunda mano pudiera tener.
En el caso de vendedores profesionales la garantía que deben ofrecer es de 12 meses. Así que rellenaremos los Nan según la legalidad que ha de ofrecer el tipo de vendedor.

In [26]:
df['Garantía'] = df.apply(lambda row: 6 if row['Tipo vendedor'] != 'Prof.' and pd.isna(row['Garantía']) else row['Garantía'], axis=1)
df['Garantía'].fillna(12, inplace=True)
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        405
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                        78
Versión del país              465
Núm. de oferta                465
Garantía                        0
Kilometraje                   569
Año                           569
Tipo de cambio                 89
Capacidad                    1323
Consumo de combustible\n2    1926
Color exterior                  0
Color original                  0
Tracción                        0
plazas                       2258
Número de marchas               0
Número de cilindros             0
Peso                            0
Tipo de combustible           986
dtype: int64

Para el consumo de combustible haremos el proceso de rellenar con los datos de misma marca y modelo, hacia alante y hacia atrás. En este caso la variable es texto por el momento, así que aplicaremos la moda cuando la copiemos de la marca o genérica. Primero le cambiaremos el nombre a la columna.

In [27]:
df=df.rename(columns={'Consumo de combustible\n2' : 'Consumo de combustible'})

In [28]:
df['Consumo de combustible'] = df.groupby(['Marca', 'Modelo'])['Consumo de combustible'].fillna(method='ffill')
df['Consumo de combustible'] = df.groupby(['Marca', 'Modelo'])['Consumo de combustible'].fillna(method='bfill')
try:
    df['Consumo de combustible'] = df.groupby(['Marca'])['Consumo de combustible'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df['Consumo de combustible'] = df['Consumo de combustible'].transform(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

Unnamed: 0                   0
Fecha extrac                 0
Enlace                       0
Marca                        0
Modelo                     405
Precio                       0
Localización                 0
Potencia                     0
Tipo vendedor                0
Categoría                    2
Tipo de vehículo             2
puertas                     78
Versión del país           465
Núm. de oferta             465
Garantía                     0
Kilometraje                569
Año                        569
Tipo de cambio              89
Capacidad                 1323
Consumo de combustible       0
Color exterior               0
Color original               0
Tracción                     0
plazas                    2258
Número de marchas            0
Número de cilindros          0
Peso                         0
Tipo de combustible        986
dtype: int64

A continuación nos ocuparemos de la capacidad. Empezaremos quitando 'cm3' después de los valores y dejando solo el número, como hicimos con los kms. Luego llenaremos según marca y modelo, luego la media de la marca, y finalmente la media total

In [29]:
df['Capacidad']=df['Capacidad'].apply(process_numbers_and_nans)

In [30]:
df['Capacidad'] = df.groupby(['Marca', 'Modelo'])['Capacidad'].fillna(method='ffill')
df['Capacidad'] = df.groupby(['Marca', 'Modelo'])['Capacidad'].fillna(method='bfill')
df['Capacidad'] = df.groupby(['Marca'])['Capacidad'].transform(lambda x: x.fillna(x.mean()))
df['Capacidad'] = df['Capacidad'].transform(lambda x: x.fillna(x.mean()))

In [31]:
df.isna().sum()

Unnamed: 0                   0
Fecha extrac                 0
Enlace                       0
Marca                        0
Modelo                     405
Precio                       0
Localización                 0
Potencia                     0
Tipo vendedor                0
Categoría                    2
Tipo de vehículo             2
puertas                     78
Versión del país           465
Núm. de oferta             465
Garantía                     0
Kilometraje                569
Año                        569
Tipo de cambio              89
Capacidad                    0
Consumo de combustible       0
Color exterior               0
Color original               0
Tracción                     0
plazas                    2258
Número de marchas            0
Número de cilindros          0
Peso                         0
Tipo de combustible        986
dtype: int64

El mismo proceso que venimos aplicando ahora para el número de plazas.

In [32]:
df['plazas'] = df.groupby(['Marca', 'Modelo'])['plazas'].fillna(method='ffill')
df['plazas'] = df.groupby(['Marca', 'Modelo'])['plazas'].fillna(method='bfill')
try:
    df['plazas'] = df.groupby(['Marca'])['plazas'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df['plazas'] = df['plazas'].transform(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    405
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   2
Tipo de vehículo            2
puertas                    78
Versión del país          465
Núm. de oferta            465
Garantía                    0
Kilometraje               569
Año                       569
Tipo de cambio             89
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                      0
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible       986
dtype: int64

Repetiremos para el tipo de combustible

In [33]:
df['Tipo de combustible'] = df.groupby(['Marca', 'Modelo'])['Tipo de combustible'].fillna(method='ffill')
df['Tipo de combustible'] = df.groupby(['Marca', 'Modelo'])['Tipo de combustible'].fillna(method='bfill')
try:
    df['Tipo de combustible'] = df.groupby(['Marca'])['Tipo de combustible'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df['Tipo de combustible'] = df['Tipo de combustible'].transform(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    405
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   2
Tipo de vehículo            2
puertas                    78
Versión del país          465
Núm. de oferta            465
Garantía                    0
Kilometraje               569
Año                       569
Tipo de cambio             89
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                      0
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
dtype: int64

Ahora lo aplicamos para el año, y luego separaremos la fecha en formato mm/aa en 'Mes' y 'Año'

In [34]:
df['Año'].value_counts()

Año
04/2023    271
05/2019    190
04/2019    178
08/2019    168
07/2019    161
          ... 
03/1998      1
11/1992      1
01/1992      1
09/1988      1
03/1987      1
Name: count, Length: 545, dtype: int64

In [35]:
df['Año'].fillna('', inplace=True)
df[['Mes', 'Año']] = df['Año'].str.split('/', expand=True)
df['Año']=df['Año'].apply(process_numbers_and_nans)

In [36]:
try:
    df['plazas'] = df.groupby(['Marca', 'Modelo'])['plazas'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df.isna().sum()


Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    405
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   2
Tipo de vehículo            2
puertas                    78
Versión del país          465
Núm. de oferta            465
Garantía                    0
Kilometraje               569
Año                       569
Tipo de cambio             89
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                    405
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
Mes                         0
dtype: int64

Para el año hemos intentado que dentro de la misma marca y el mismo modelo nos buscara la moda del año que más hay de ese modelo, pero no nos ha servido para disminuir el número de NaNs. En este caso no parece buena idea coger la media de la marca o general porque las mismas marcas llevan fabricando coches muchas décadas y una mala asignación del año de un vehículo en el rango de décadas afectaría mucho a la predicción. Así que eliminaremos los registros de los que no podemos obtener el año.

In [37]:
df=df[df['Año'].notna()]
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    378
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   0
Tipo de vehículo            0
puertas                    76
Versión del país          463
Núm. de oferta            463
Garantía                    0
Kilometraje                 0
Año                         0
Tipo de cambio             87
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                    378
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
Mes                         0
dtype: int64

Pasamos al atributo 'Version del país'. El único valor disponible en el dataset es España, así que es el que asignaremos a todos los vehículos con Nan en ese atributo

In [38]:
df['Versión del país'].value_counts()

Versión del país
España    10141
Name: count, dtype: int64

In [39]:
df['Versión del país'].fillna('España', inplace=True)
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    378
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   0
Tipo de vehículo            0
puertas                    76
Versión del país            0
Núm. de oferta            463
Garantía                    0
Kilometraje                 0
Año                         0
Tipo de cambio             87
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                    378
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
Mes                         0
dtype: int64

El siguiente atributo con NaNs es 'Núm. de oferta'. Este atributo al ser un identificador nos sirve para detectar duplicados, pero una vez se han eliminado estos no tiene interés para el análisis ni el modelo, así que lo eliminaremos.

In [40]:
df['Núm. de oferta'].value_counts()

Núm. de oferta
10506192.0    2
11062636.0    2
9787939.0     2
10794610.0    2
10889853.0    2
             ..
8189853.0     1
11136647.0    1
8906401.0     1
8559553.0     1
10752433.0    1
Name: count, Length: 10136, dtype: int64

In [41]:
df[df['Núm. de oferta']==10506192.0]

,Unnamed: 0,Fecha extrac,Enlace,Marca,Modelo,Precio,Localización,Potencia,Tipo vendedor,Categoría,...,Consumo de combustible,Color exterior,Color original,Tracción,plazas,Número de marchas,Número de cilindros,Peso,Tipo de combustible,Mes
360,360,2023-11-11,https://www.autoscout24.es/anuncios/audi-q7-4-...,Audi,Q7,"€ 64.623,-",https://maps.google.com/?q=C%2F%20MATRICERS%20...,320 kW (435 CV),Prof.,SUV/4x4/Pickup,...,0 l/100 km (mixto)\n0 l/100 km (urbano)\n0 l/1...,Blanco,Blanco,Tracción delantera,5.0,6,4,1165.0,Diésel,03
1539,1539,2023-11-13,https://www.autoscout24.es/anuncios/audi-q7-s-...,Audi,Q7,"€ 60.912,-",https://maps.google.com/?q=C%2F%20MATRICERS%20...,320 kW (435 CV),Prof.,SUV/4x4/Pickup,...,"5 l/100 km (mixto)\n5,9 l/100 km (urbano)\n4,6...",Blanco,Blanco,Tracción delantera,5.0,6,4,1460.0,Diésel,03


Vemos que efectivamente cuando el número de oferta coincide el vehículo anunciado es el mismo. Eliminaremos estos duplicados y posteriormente la columna.

In [42]:
df=df.drop_duplicates(['Núm. de oferta'], keep='last')

In [43]:
df['Núm. de oferta'].value_counts()

Núm. de oferta
11099248.0    1
10997710.0    1
9232695.0     1
11344210.0    1
9821752.0     1
             ..
8189853.0     1
11136647.0    1
8906401.0     1
8559553.0     1
10752433.0    1
Name: count, Length: 10136, dtype: int64

In [44]:
df=df.drop(['Núm. de oferta'], axis=1)
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    349
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   0
Tipo de vehículo            0
puertas                    25
Versión del país            0
Garantía                    0
Kilometraje                 0
Año                         0
Tipo de cambio              7
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                    349
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
Mes                         0
dtype: int64

Vamos a aplicar el proceso que venimos haciendo al atributo 'plazas'

In [45]:
df['plazas'] = df.groupby(['Marca', 'Modelo'])['plazas'].fillna(method='ffill')
df['plazas'] = df.groupby(['Marca', 'Modelo'])['plazas'].fillna(method='bfill')
try:
    df['plazas'] = df.groupby(['Marca'])['plazas'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df['plazas'] = df['plazas'].transform(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    349
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   0
Tipo de vehículo            0
puertas                    25
Versión del país            0
Garantía                    0
Kilometraje                 0
Año                         0
Tipo de cambio              7
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                      0
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
Mes                         0
dtype: int64

Los NaN en el atributo 'Modelo' son porque no se ha especificado el modelo en la casilla correspondiente al poner el anuncio. A veces se puede encontrar en la descripción, pero dado que son pocos no compensa el trabajo de intentar encontrarlo allí. Puesto que el modelo es un atributo muy importante a la hora de calcular el precio, tampoco parece razonable imputarlo a través de la moda o similares. Así que borraremos los registros que no tienen el modelo especificado.

In [46]:
df=df[df['Modelo'].notna()]
df.isna().sum()

Unnamed: 0                 0
Fecha extrac               0
Enlace                     0
Marca                      0
Modelo                     0
Precio                     0
Localización               0
Potencia                   0
Tipo vendedor              0
Categoría                  0
Tipo de vehículo           0
puertas                   21
Versión del país           0
Garantía                   0
Kilometraje                0
Año                        0
Tipo de cambio             6
Capacidad                  0
Consumo de combustible     0
Color exterior             0
Color original             0
Tracción                   0
plazas                     0
Número de marchas          0
Número de cilindros        0
Peso                       0
Tipo de combustible        0
Mes                        0
dtype: int64

Finalmente solo nos quedan NaN en el atributo 'puertas'. Usaremos el método de copiarlo de coches de la misma marca y modelo o coger la moda de la marca o general.

In [47]:
df['puertas'] = df.groupby(['Marca', 'Modelo'])['puertas'].fillna(method='ffill')
df['puertas'] = df.groupby(['Marca', 'Modelo'])['puertas'].fillna(method='bfill')
try:
    df['puertas'] = df.groupby(['Marca'])['puertas'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df['puertas'] = df['puertas'].transform(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

Unnamed: 0                0
Fecha extrac              0
Enlace                    0
Marca                     0
Modelo                    0
Precio                    0
Localización              0
Potencia                  0
Tipo vendedor             0
Categoría                 0
Tipo de vehículo          0
puertas                   0
Versión del país          0
Garantía                  0
Kilometraje               0
Año                       0
Tipo de cambio            6
Capacidad                 0
Consumo de combustible    0
Color exterior            0
Color original            0
Tracción                  0
plazas                    0
Número de marchas         0
Número de cilindros       0
Peso                      0
Tipo de combustible       0
Mes                       0
dtype: int64

In [48]:
df.head(10)

,Unnamed: 0,Fecha extrac,Enlace,Marca,Modelo,Precio,Localización,Potencia,Tipo vendedor,Categoría,...,Consumo de combustible,Color exterior,Color original,Tracción,plazas,Número de marchas,Número de cilindros,Peso,Tipo de combustible,Mes
0,0,2023-11-11,https://www.autoscout24.es/anuncios/lynk-co-01...,Lynk & Co,01,"€ 22.630,-1",https://maps.google.com/?q=AVENIDA%20CARLOS%20...,145 kW (197 CV),Prof.,SUV/4x4/Pickup,...,0 l/100 km (mixto)\n0 l/100 km (urbano)\n0 l/1...,Azul,Azul,Tracción delantera,5.0,6,3,1237.0,Gasolina,11
1,1,2023-11-11,https://www.autoscout24.es/anuncios/cupra-form...,Cupra,Formentor,"€ 20.995,-",https://maps.google.com/?q=AVENIDA%20DE%20CATA...,110 kW (150 CV),Prof.,SUV/4x4/Pickup,...,0 l/100 km (mixto)\n0 l/100 km (urbano)\n0 l/1...,Negro,Negro Midnight (metalizado),Tracción a las cuatro ruedas,5.0,7,4,1761.0,Diésel,05
2,2,2023-11-11,https://www.autoscout24.es/anuncios/gmc-yukon-...,GMC,Yukon,"€ 79.000,-",https://maps.google.com/?q=C%2F%20VICENTE%20MU...,313 kW (426 CV),Prof.,SUV/4x4/Pickup,...,"5,1 l/100 km (mixto)\n5,3 l/100 km (urbano)\n4...",Gris,Gris,Tracción a las cuatro ruedas,2.0,8,4,1760.0,Diésel,06
3,3,2023-11-11,https://www.autoscout24.es/anuncios/infiniti-q...,Infiniti,Q50,"€ 27.250,-",https://maps.google.com/?q=CARRETERA%20N-II%20...,268 kW (364 CV),Prof.,Sedán,...,8 l/100 km (mixto)\n10 l/100 km (urbano)\n7 l/...,ROJO,ROJO,Tracción a las cuatro ruedas,4.0,8,8,2215.0,Diésel,01
4,4,2023-11-11,https://www.autoscout24.es/anuncios/audi-r8-4-...,Audi,R8,"€ 51.500,-",https://maps.google.com/?q=Ctra.%20Madrid%2C%2...,309 kW (420 CV),Prof.,Coupé,...,"5,3 l/100 km (mixto)\n6,1 l/100 km (urbano)\n4...",Plateado,Gris Plata,Tracción delantera,5.0,6,4,1580.0,Diésel,06
5,5,2023-11-11,https://www.autoscout24.es/anuncios/porsche-99...,Porsche,992,"€ 289.900,-",https://maps.google.com/?q=Poligono%20Industri...,478 kW (650 CV),Prof.,Coupé,...,"5,7 l/100 km (mixto)\n7 l/100 km (urbano)\n5 l...",Gris,Gris Ágata,Tracción delantera,5.0,5,4,1285.0,Gasolina,11
6,6,2023-11-11,https://www.autoscout24.es/anuncios/ferrari-81...,Ferrari,812,"€ 379.999,-",https://maps.google.com/?q=C%2FCARRIL%20DE%20P...,588 kW (799 CV),Prof.,Coupé,...,0 l/100 km (mixto)\n0 l/100 km (urbano)\n0 l/1...,Rojo,Rosso Corsa,Tracción trasera,5.0,9,4,1870.0,Diésel,01
7,7,2023-11-11,https://www.autoscout24.es/anuncios/ds-automob...,DS Automobiles,DS 7 Crossback,"€ 21.290,-1",https://maps.google.com/?q=AV.%20DE%20AR%C3%93...,96 kW (131 CV),Prof.,SUV/4x4/Pickup,...,0 l/100 km (mixto)\n0 l/100 km (urbano)\n0 l/1...,Negro,Negro,Tracción trasera,5.0,7,10,1339.0,Gasolina,03
8,8,2023-11-11,https://www.autoscout24.es/anuncios/bentley-co...,Bentley,Continental,"€ 64.500,-",https://maps.google.com/?q=Crtra.de%20Cadiz%20...,449 kW (610 CV),Prof.,Coupé,...,"6 l/100 km (mixto)\n7,8 l/100 km (urbano)\n4,9...",Blanco,Blanco,Tracción trasera,5.0,8,6,1950.0,Diésel,06
9,9,2023-11-11,https://www.autoscout24.es/anuncios/mercedes-b...,Mercedes-Benz,GLC 250,"€ 35.900,-1",https://maps.google.com/?q=AVDA%20CAMINO%20DE%...,150 kW (204 CV),Prof.,SUV/4x4/Pickup,...,"9,1 l/100 km (mixto)\n12,3 l/100 km (urbano)\n...",Azul,PACK AMG,Tracción trasera,4.0,6,4,935.0,Gasolina,05


Ahora que ya hemos tratado los NaN iremos a limpiar los problemas de formato. Lo primero que haremos es eliminar la columna 'Unnamed: 0' que equivale al índice. Y a continuación quitaremos el símbolo de euro y el punto al atributo 'Precio'

In [49]:
df=df.drop(['Unnamed: 0'], axis=1)

In [50]:
df['Precio']=df['Precio'].apply(process_numbers_and_nans)

Del atributo 'Potencia' nos quedaremos con los caballos (CV) ya que es la medida que se usa en España.

In [51]:
df['Potencia']=df['Potencia'].str.extract(r'\((\d+)\s*CV\)')
df['Potencia'].head()

0    197
1    150
2    426
3    364
4    420
Name: Potencia, dtype: object

Al atributo 'Kilometraje' hay que quitarle el 'km' después del número y el punto de separación de miles.

In [53]:
df['Kilometraje']=df['Kilometraje'].apply(process_numbers_and_nans)
df['Kilometraje'].head()

0    53627
1    26141
2    81422
3    92253
4    81100
Name: Kilometraje, dtype: int64

El atributo 'Consumo de combustible' contiene en realidad 3 datos (consumo mixto, urbano y extraurbano). Nos quedaremos solo con el consumo mixto y únicamente con la cifra

In [56]:
df['Consumo de combustible'][2]

'5,1 l/100 km (mixto)\n5,3 l/100 km (urbano)\n4,9 l/100 km (extraurbano)'

In [57]:
df['Consumo de combustible'] = df['Consumo de combustible'].str.extract(r'(\d+,\d+)')

In [58]:
df['Consumo de combustible'].head()

0    NaN
1    NaN
2    5,1
3    NaN
4    5,3
Name: Consumo de combustible, dtype: object

La limpieza del atributo ha generado los Nan que estaban ocultos por estar puesto a valor 0 (ningún coche tiene un consumo 0). Vamos a ver cuántos NaN tenemos y a limpiarlos con el proceso que hemos seguido en la mayoría de campos.

In [59]:
df['Consumo de combustible'].isna().sum()

2577

In [66]:
df['Consumo de combustible']=df['Consumo de combustible'].str.replace(',', '.')
df['Consumo de combustible']=df['Consumo de combustible'].astype('float64')

In [71]:
df['Consumo de combustible'] = df.groupby(['Marca', 'Modelo'])['Consumo de combustible'].fillna(method='ffill')
df['Consumo de combustible'] = df.groupby(['Marca', 'Modelo'])['Consumo de combustible'].fillna(method='bfill')
try:
    df['Consumo de combustible'] = df.groupby(['Marca'])['Consumo de combustible'].transform(lambda x: x.fillna(x.mean()))
except:
    pass
df['Consumo de combustible'] = df['Consumo de combustible'].transform(lambda x: x.fillna(x.mean()))
df['Consumo de combustible'].isna().sum()

0

A continuación vamos a ver qué columnas no están en el formato que nos interesa y se lo cambiaremos

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9788 entries, 0 to 11172
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Fecha extrac            9788 non-null   object 
 1   Enlace                  9788 non-null   object 
 2   Marca                   9788 non-null   object 
 3   Modelo                  9788 non-null   object 
 4   Precio                  9788 non-null   int64  
 5   Localización            9788 non-null   object 
 6   Potencia                9532 non-null   object 
 7   Tipo vendedor           9788 non-null   object 
 8   Categoría               9788 non-null   object 
 9   Tipo de vehículo        9788 non-null   object 
 10  puertas                 9788 non-null   float64
 11  Versión del país        9788 non-null   object 
 12  Garantía                9788 non-null   object 
 13  Kilometraje             9788 non-null   int64  
 14  Año                     9788 non-null   floa

In [76]:
df[['puertas', 'Garantía', 'Año', 'plazas', 'Mes']]=df[['puertas', 'Garantía', 'Año', 'plazas', 'Mes']].astype('int64')
df['Precio']=df['Precio'].astype('float64')

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9788 entries, 0 to 11172
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Fecha extrac            9788 non-null   object 
 1   Enlace                  9788 non-null   object 
 2   Marca                   9788 non-null   object 
 3   Modelo                  9788 non-null   object 
 4   Precio                  9788 non-null   float64
 5   Localización            9788 non-null   object 
 6   Potencia                9532 non-null   object 
 7   Tipo vendedor           9788 non-null   object 
 8   Categoría               9788 non-null   object 
 9   Tipo de vehículo        9788 non-null   object 
 10  puertas                 9788 non-null   int64  
 11  Versión del país        9788 non-null   object 
 12  Garantía                9788 non-null   int64  
 13  Kilometraje             9788 non-null   int64  
 14  Año                     9788 non-null   int6

In [78]:
df.head()

,Fecha extrac,Enlace,Marca,Modelo,Precio,Localización,Potencia,Tipo vendedor,Categoría,Tipo de vehículo,...,Consumo de combustible,Color exterior,Color original,Tracción,plazas,Número de marchas,Número de cilindros,Peso,Tipo de combustible,Mes
0,2023-11-11,https://www.autoscout24.es/anuncios/lynk-co-01...,Lynk & Co,01,226301.0,https://maps.google.com/?q=AVENIDA%20CARLOS%20...,197,Prof.,SUV/4x4/Pickup,Ocasión,...,8.3,Azul,Azul,Tracción delantera,5,6,3,1237.0,Gasolina,11
1,2023-11-11,https://www.autoscout24.es/anuncios/cupra-form...,Cupra,Formentor,20995.0,https://maps.google.com/?q=AVENIDA%20DE%20CATA...,150,Prof.,SUV/4x4/Pickup,Ocasión,...,12.1,Negro,Negro Midnight (metalizado),Tracción a las cuatro ruedas,5,7,4,1761.0,Diésel,5
2,2023-11-11,https://www.autoscout24.es/anuncios/gmc-yukon-...,GMC,Yukon,79000.0,https://maps.google.com/?q=C%2F%20VICENTE%20MU...,426,Prof.,SUV/4x4/Pickup,Ocasión,...,5.1,Gris,Gris,Tracción a las cuatro ruedas,2,8,4,1760.0,Diésel,6
3,2023-11-11,https://www.autoscout24.es/anuncios/infiniti-q...,Infiniti,Q50,27250.0,https://maps.google.com/?q=CARRETERA%20N-II%20...,364,Prof.,Sedán,Ocasión,...,2.7,ROJO,ROJO,Tracción a las cuatro ruedas,4,8,8,2215.0,Diésel,1
4,2023-11-11,https://www.autoscout24.es/anuncios/audi-r8-4-...,Audi,R8,51500.0,https://maps.google.com/?q=Ctra.%20Madrid%2C%2...,420,Prof.,Coupé,Ocasión,...,5.3,Plateado,Gris Plata,Tracción delantera,5,6,4,1580.0,Diésel,6


Sobre la localización, actualmente tenemos un enlace de Google Maps, que nos puede ser muy útil, pero nos gustaría tener también un atributo 'Província'. Esa información está contenida en el enlace de Google Maps, así que vamos a parsearlo para obtenerla

In [92]:
def obtener_ciudad_cp(url):
    parsed_url = urlparse(url)
    query_params = dict(pair.split('=') for pair in parsed_url.query.split('&'))
    
    # Decodificar la dirección de la URL
    decoded_address = unquote(query_params['q'])
    
    # Dividir la dirección en partes
    partes_direccion = decoded_address.split(',')
    
    # Obtener la ciudad y el código postal
    cp_ciudad = partes_direccion[-2].strip()
    #La ciudad contiene actualmente el código postal, vamos a separlo en dos atributos
    cp, ciudad = cp_ciudad.split(' ', 1) if ' ' in cp_ciudad else (cp_ciudad, '')
    ciudad=ciudad.capitalize()

    return ciudad, cp

In [94]:
df[['Ciudad', 'CP']] = df['Localización'].apply(lambda x: pd.Series(obtener_ciudad_cp(x)))
df[['Localización','Ciudad', 'CP']].head(10)

,Localización,Ciudad,CP
0,https://maps.google.com/?q=AVENIDA%20CARLOS%20...,Madrid,28914
1,https://maps.google.com/?q=AVENIDA%20DE%20CATA...,Zaragoza,50014
2,https://maps.google.com/?q=C%2F%20VICENTE%20MU...,Madrid,28043
3,https://maps.google.com/?q=CARRETERA%20N-II%20...,Fornells de la selva,17458
4,https://maps.google.com/?q=Ctra.%20Madrid%2C%2...,Alicante,3006
5,https://maps.google.com/?q=Poligono%20Industri...,Cardedeu,8440
6,https://maps.google.com/?q=C%2FCARRIL%20DE%20P...,Marbella,29670
7,https://maps.google.com/?q=AV.%20DE%20AR%C3%93...,Pamplona,31009
8,https://maps.google.com/?q=Crtra.de%20Cadiz%20...,San pedro de alcantara,29670
9,https://maps.google.com/?q=AVDA%20CAMINO%20DE%...,San sebastian de los reyes,28703


Ya tenemos la ciudad y el código postal, pero nos gustaría conocer también la provincia. La obtendremos con geopy

In [123]:
#Abandonar toda esperanza y usar el xls de códigos postales
def obtener_provincia(ciudad):
    geo=Nominatim(user_agent="BuscaProv")
    location=geo.geocode(ciudad)
    #lista_loc=loc[0].split(",")
    print(location)

In [124]:
obtener_provincia('Cardedeu')
obtener_provincia('Zaragoza')
obtener_provincia('Pamplona')
obtener_provincia('Alicante')

Cardedeu, Vallès Oriental, Barcelona, Catalunya, 08440, España
Zaragoza, Aragón, España
Pamplona/Iruña, Iruñerria / Comarca de Pamplona, España
Alacant / Alicante, l'Alacantí, Alacant / Alicante, Comunitat Valenciana, España


Ya tenemos el primer dataset limpio, vamos a guardarlo en un archivo diferente para usarlo para el análisis y el modelo.

In [79]:
df.to_csv('veh_usados_clean.csv')